## Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd()

In [1]:
import numpy as np
from sklearn import datasets

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter('ignore')

In [2]:
# Загрузим игрушечный датасет из sklearn
iris = datasets.load_iris()
X = iris.data
X.shape

(150, 4)

In [3]:
# Для начала отмасштабируем выборку
X_ = X.astype(float)

rows, cols = X_.shape

# центрирование - вычитание из каждого значения среднего по строке
means = X_.mean(0)
for i in range(rows):
    for j in range(cols):
        X_[i, j] -= means[j]
        
# деление каждого значения на стандартное отклонение
std = np.std(X_, axis=0)
for i in range(cols):
    for j in range(rows):
        X_[j][i] /= std[i]

In [4]:
U, D, V = np.linalg.svd(X_)

In [5]:
U.shape, D.shape, V.shape

((150, 150), (4,), (4, 4))

In [6]:
eig_sum = sum(D)
var_exp = [(i / eig_sum) * 100 for i in D]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описываемая каждой из компонент \n{var_exp}')

# а теперь оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описываемая каждой из компонент 
[53.52971788236243, 29.956812784020066, 12.003682417209491, 4.509786916408036]
Кумулятивная доля дисперсии по компонентам 
[ 53.52971788  83.48653067  95.49021308 100.        ]


In [7]:
# последняя компонента описывает 4.5% информации, поэтому её можно отбросить
W = V[:, :3]
W

array([[ 0.52106591, -0.26934744,  0.5804131 ],
       [-0.37741762, -0.92329566, -0.02449161],
       [ 0.71956635, -0.24438178, -0.14212637],
       [ 0.26128628, -0.12350962, -0.80144925]])

In [8]:
# новая матрица
X_new = X_ @ W
X_new

array([[-2.16199391, -0.20824916,  0.69701917],
       [-1.85386523,  0.91972173,  0.58455377],
       [-2.19479706,  0.57380492,  0.44070126],
       [-2.08925659,  0.79120079,  0.35985553],
       [-2.31201067, -0.38815258,  0.62105386],
       [-2.12862356, -1.2305024 ,  0.65022254],
       [-2.35640215,  0.15121396,  0.24552345],
       [-2.09735174,  0.02303674,  0.62425154],
       [-2.08266715,  1.29544269,  0.23855457],
       [-1.93424093,  0.70954986,  0.6763343 ],
       [-2.1054469 , -0.74512731,  0.88864755],
       [-2.18272634,  0.07441922,  0.47551861],
       [-1.95139536,  0.96861583,  0.61972287],
       [-2.3897725 ,  1.1734672 ,  0.29232007],
       [-2.23623695, -1.47162126,  1.17727785],
       [-2.45541239, -2.36332948,  0.84917155],
       [-2.29221676, -1.17494228,  0.68253493],
       [-2.12760023, -0.224507  ,  0.59152265],
       [-1.88672759, -1.09961365,  0.97233921],
       [-2.34734287, -0.87601606,  0.56653089],
       [-1.76300935, -0.13528835,  0.889

## Обучить любую модель классификации на датасете IRIS до применения PCA и после него. Сравнить качество классификации по отложенной выборке.

Загрука и подготовка данных

In [9]:
X, y = datasets.load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = 5
)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Обучение модели до применения PCA

In [10]:
lr = LogisticRegression()

lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)

print(f'Точность модели на тренировочной выборке {accuracy_score(y_train, y_pred_train)}')
print(f'Точность модели на тестовой выборке {accuracy_score(y_test, y_pred_test)}')

Точность модели на тренировочной выборке 0.9809523809523809
Точность модели на тестовой выборке 0.9777777777777777


Обучение модели после применения PCA

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X_new, y, test_size = 0.3, random_state = 5
)

lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)

print(f'Точность модели на тренировочной выборке {accuracy_score(y_train, y_pred_train)}')
print(f'Точность модели на тестовой выборке {accuracy_score(y_test, y_pred_test)}')

Точность модели на тренировочной выборке 0.9714285714285714
Точность модели на тестовой выборке 0.9555555555555556
